In [1]:
import numpy as np
import pandas as pd

import matplotlib as plt
import seaborn as sns
import time

from google.oauth2.service_account import Credentials
import logging

from oauth2client.service_account import ServiceAccountCredentials
from datetime import date, timedelta
import df2gspread.df2gspread as d2g
from apiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
import logging
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [3]:
credentials=Credentials.from_service_account_file('C:/Users/mncedisi.mncwabe/Documents/Connect to BQ/tal-production-data-4390ad039b75.json')

In [4]:
plids = pd.read_excel(r"C:/Users/mncedisi.mncwabe/Downloads/Retail_overlap_tsins_ with_MP.xlsx")

In [24]:
plids.head()

,idTsin,idProductline,Overlap
0,3,8575,1
1,5,8577,1
2,7,8579,1
3,9,8581,1
4,11,8583,1


In [25]:
plids['idProductline'].nunique()

448378

In [17]:
from tqdm import tqdm

In [19]:
start_time = time.time()

idproductlist = list(plids.idProductline.unique())
#convert the list to a tuple
p = tuple(idproductlist)
chunk = 10000
plids_df = []
for i in tqdm(range(0, len(p), chunk)):
    querysch = f'''SELECT
       COUNT(bb_winner.payload.data.product.product_line_id) AS freq,
       FORMAT_DATE("%Y-%m",DATETIME(bb_winner.timestamp,'Africa/Johannesburg')) AS Month,
     CASE 
         WHEN bb_winner.payload.data.product.market_place_listing = true THEN "Marketplace"
         ELSE "Retail"
     END AS buybox_winner
  FROM `gcp-takealot.prod_user_tracking.impression_action` AS bb_winner
  WHERE 
   -- 'product_details' contexts contain the buybox winner
     bb_winner.payload.event_timestamp >= "2020-10-01"  AND bb_winner.payload.event_timestamp < "2021-11-01"
   AND bb_winner.timestamp >= "2020-10-01" AND bb_winner.timestamp < "2021-11-01"
   AND DATETIME(bb_winner.timestamp,'Africa/Johannesburg') >= '2020-10-01T00:00:00.0000'
   AND DATETIME(bb_winner.timestamp,'Africa/Johannesburg') < '2021-11-01T00:00:00.0000'
   AND bb_winner.payload.context = "product_details"
   AND bb_winner.payload.data.product.market_place_listing IS NOT NULL
   AND bb_winner.payload.data.product.product_line_id IN {p[i:i + chunk]}
   GROUP BY 
       buybox_winner,
       Month
       '''
    df = pd.read_gbq(querysch, project_id='gcp-takealot', dialect='standard', credentials=credentials)
    plids_df.append(df)

plidsDF = pd.concat(plids_df)

  0%|          | 0/45 [00:00<?, ?it/s]Requesting query... 
2021-12-22 15:16:54,487 : DEBUG : Requesting query... 
Query running...
2021-12-22 15:16:56,794 : DEBUG : Query running...
Job ID: ab577dd5-d378-47fd-b4b7-a9c5b82ea257
2021-12-22 15:16:56,797 : DEBUG : Job ID: ab577dd5-d378-47fd-b4b7-a9c5b82ea257
  Elapsed 6.74 s. Waiting...
2021-12-22 15:17:01,231 : INFO :   Elapsed 6.74 s. Waiting...
  Elapsed 7.84 s. Waiting...
2021-12-22 15:17:02,326 : INFO :   Elapsed 7.84 s. Waiting...
  Elapsed 8.96 s. Waiting...
2021-12-22 15:17:03,450 : INFO :   Elapsed 8.96 s. Waiting...
  Elapsed 10.05 s. Waiting...
2021-12-22 15:17:04,539 : INFO :   Elapsed 10.05 s. Waiting...
  Elapsed 11.18 s. Waiting...
2021-12-22 15:17:05,667 : INFO :   Elapsed 11.18 s. Waiting...
  Elapsed 12.34 s. Waiting...
2021-12-22 15:17:06,830 : INFO :   Elapsed 12.34 s. Waiting...
  Elapsed 13.75 s. Waiting...
2021-12-22 15:17:08,237 : INFO :   Elapsed 13.75 s. Waiting...
  Elapsed 14.84 s. Waiting...
2021-12-22 15:17:09

In [22]:
plidsDF.head()

,freq,Month,buybox_winner
0,968,2020-12,Retail
1,930,2021-02,Marketplace
2,664,2021-09,Marketplace
3,751,2020-11,Marketplace
4,652,2020-10,Marketplace


In [21]:
plidsDF.shape

(1166, 3)

In [23]:
plidsDF.to_csv("C:/Users/mncedisi.mncwabe/Downloads/Buybox Competition Comms/plids.csv")

In [1]:
from tqdm import tqdm

In [28]:
start_time = time.time()

idproductlistv = list(plids.idProductline.unique())
#convert the list to a tuple
t = tuple(idproductlistv)
chunkv = 50000
p_df = []
for i in tqdm(range(0, len(t), chunkv)):
    query = f'''SELECT
       COUNT(bb_winner.payload.data.product.product_line_id) AS freq,
       FORMAT_DATE("%Y-%m",DATETIME(bb_winner.timestamp,'Africa/Johannesburg')) AS Month,
     CASE 
         WHEN bb_winner.payload.data.product.market_place_listing = true THEN "Marketplace"
         ELSE "Retail"
     END AS buybox_winner
  FROM `gcp-takealot.prod_user_tracking.impression_action` AS bb_winner
  WHERE 
   -- 'product_details' contexts contain the buybox winner
     bb_winner.payload.event_timestamp >= "2020-10-01"  AND bb_winner.payload.event_timestamp < "2021-11-01"
   AND bb_winner.timestamp >= "2020-10-01" AND bb_winner.timestamp < "2021-11-01"
   AND DATETIME(bb_winner.timestamp,'Africa/Johannesburg') >= '2020-10-01T00:00:00.0000'
   AND DATETIME(bb_winner.timestamp,'Africa/Johannesburg') < '2021-11-01T00:00:00.0000'
   AND bb_winner.payload.context = "product_details"
   AND bb_winner.payload.data.product.market_place_listing IS NOT NULL
   AND bb_winner.payload.data.product.product_line_id IN {t[i:i + chunkv]}
   GROUP BY 
       buybox_winner,
       Month
       '''
    dfv = pd.read_gbq(query, project_id='gcp-takealot', dialect='standard', credentials=credentials)
    p_df.append(dfv)

p_df = pd.concat(p_df)

  0%|          | 0/9 [00:00<?, ?it/s]Requesting query... 
2021-12-22 21:34:01,905 : DEBUG : Requesting query... 
Query running...
2021-12-22 21:34:09,124 : DEBUG : Query running...
Job ID: e334b950-0f93-42d5-adbc-9fdf631fd591
2021-12-22 21:34:09,124 : DEBUG : Job ID: e334b950-0f93-42d5-adbc-9fdf631fd591
  Elapsed 7.22 s. Waiting...
2021-12-22 21:34:09,124 : INFO :   Elapsed 7.22 s. Waiting...
  Elapsed 8.32 s. Waiting...
2021-12-22 21:34:10,224 : INFO :   Elapsed 8.32 s. Waiting...
  Elapsed 9.4 s. Waiting...
2021-12-22 21:34:11,304 : INFO :   Elapsed 9.4 s. Waiting...
  Elapsed 10.51 s. Waiting...
2021-12-22 21:34:12,414 : INFO :   Elapsed 10.51 s. Waiting...
  Elapsed 11.64 s. Waiting...
2021-12-22 21:34:13,546 : INFO :   Elapsed 11.64 s. Waiting...
  Elapsed 12.77 s. Waiting...
2021-12-22 21:34:14,672 : INFO :   Elapsed 12.77 s. Waiting...
  Elapsed 13.87 s. Waiting...
2021-12-22 21:34:15,779 : INFO :   Elapsed 13.87 s. Waiting...
  Elapsed 14.94 s. Waiting...
2021-12-22 21:34:16,85

In [29]:
p_df.head()

,freq,Month,buybox_winner
0,1280477,2021-10,Retail
1,288755,2021-10,Marketplace
2,253443,2020-10,Marketplace
3,1311364,2020-10,Retail
4,1685144,2021-01,Retail


In [30]:
p_df.to_csv("C:/Users/mncedisi.mncwabe/Downloads/Buybox Competition Comms/p_df.csv")